## Motivation

Although many cost functions are more than fast enough already, I wanted to a simple pythonic version, while still being blazing fast. The results are somewhere between 23 and 24 µs.

Also, I updated the cost function so it reports how many high and low days there are, instead of only out of bounds. The original function returns 4 values: family cost, accounting cost, number of days under 125 or over 300. I also show how to use closures to combine costs and penalties for out of bounds days. 

## References:

- Original Kernel: https://www.kaggle.com/inversion/santa-s-2019-starter-notebook
- First kernel that had the idea to use Numba: https://www.kaggle.com/nickel/250x-faster-cost-function-with-numba-jit
- Another great cost function optimization: https://www.kaggle.com/sekrier/fast-scoring-using-c-52-usec
- Original cost matrix idea: https://www.kaggle.com/paulorzp/cost-matrix-low-memory-and-fast
- Better performance with family cost matrix idea: https://www.kaggle.com/xhlulu/santa-s-2019-300x-faster-cost-function-37-s
- Better performance with accounting cost matrix idea: https://www.kaggle.com/xhlulu/santa-s-2019-faster-cost-function-24-s
- Submission fle from: https://www.kaggle.com/xhlulu/santa-s-2019-stochastic-product-search

In [1]:
import numpy as np
import pandas as pd
from numba import njit, prange

In [17]:
dir_linux = !ls -a
dir_win = !dir /b
dir_all = list(dir_linux) + list(dir_win)
if 'kernel-metadata.json' in dir_all:
    src_local = True
    print('Local')
else:
    src_local = False
    print('Kaggle')

if src_local:
    data_folder = '../../../data/santa-workshop-tour-2019/'
else:
    data_folder = '../input/santa-2019-workshop-scheduling/'
fpath = data_folder + 'family_data.csv'
data = pd.read_csv(fpath, index_col='family_id')

fpath = data_folder + 'sample_submission.csv'
prediction = pd.read_csv(fpath, index_col='family_id')

Local


In [18]:
family_size = data.n_people.values.astype(np.int8)

In [19]:
data.n_people.values

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

In [26]:
penalties

array([[   0,   50,   50,  100,  200,  200,  300,  300,  400,  500,  500],
       [   0,   50,   59,  109,  209,  218,  318,  336,  436,  735,  934],
       [   0,   50,   68,  118,  218,  236,  336,  372,  472,  970, 1368],
       [   0,   50,   77,  127,  227,  254,  354,  408,  508, 1205, 1802],
       [   0,   50,   86,  136,  236,  272,  372,  444,  544, 1440, 2236],
       [   0,   50,   95,  145,  245,  290,  390,  480,  580, 1675, 2670],
       [   0,   50,  104,  154,  254,  308,  408,  516,  616, 1910, 3104],
       [   0,   50,  113,  163,  263,  326,  426,  552,  652, 2145, 3538],
       [   0,   50,  122,  172,  272,  344,  444,  588,  688, 2380, 3972]])

In [16]:
penalties = np.asarray([
    [
        0,
        50,
        50 + 9 * n,
        100 + 9 * n,
        200 + 9 * n,
        200 + 18 * n,
        300 + 18 * n,
        300 + 36 * n,
        400 + 36 * n,
        500 + 36 * n + 199 * n,
        500 + 36 * n + 398 * n
    ] for n in range(family_size.max() + 1)
])
family_cost_matrix = np.concatenate(data.n_people.apply(lambda n: np.repeat(penalties[n, 10], 100).reshape(1, 100)))
for fam in data.index:
    for choice_order, day in enumerate(data.loc[fam].drop("n_people")):
        family_cost_matrix[fam, day - 1] = penalties[data.loc[fam, "n_people"], choice_order]

In [4]:
accounting_cost_matrix = np.zeros((500, 500))
for n in range(accounting_cost_matrix.shape[0]):
    for diff in range(accounting_cost_matrix.shape[1]):
        accounting_cost_matrix[n, diff] = max(0, (n - 125.0) / 400.0 * n**(0.5 + diff / 50.0))


In [5]:
@njit(fastmath=True)
def cost_function(prediction, family_size, family_cost_matrix, accounting_cost_matrix):
    N_DAYS = family_cost_matrix.shape[1]
    MAX_OCCUPANCY = 300
    MIN_OCCUPANCY = 125
    penalty = 0
    daily_occupancy = np.zeros(N_DAYS + 1, dtype=np.int16)
    for i, (pred, n) in enumerate(zip(prediction, family_size)):
        daily_occupancy[pred - 1] += n
        penalty += family_cost_matrix[i, pred - 1]

    accounting_cost = 0
    n_low = 0
    n_high = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        n_next = daily_occupancy[day + 1]
        n = daily_occupancy[day]
        n_high += (n > MAX_OCCUPANCY) 
        n_low += (n < MIN_OCCUPANCY)
        diff = abs(n - n_next)
        accounting_cost += accounting_cost_matrix[n, diff]

    return np.asarray([penalty, accounting_cost, n_low, n_high])

In [6]:
family_size = family_size.astype(np.int16)

In [7]:
cost_function(prediction, family_size, family_cost_matrix, accounting_cost_matrix)

array([71340.        ,  5906.77453972,     0.        ,     0.        ])

In [8]:
%timeit cost_function(prediction, family_size, family_cost_matrix, accounting_cost_matrix)

23.5 µs ± 267 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
def get_cost_consolidated(prediction): 
    fc, ac, l, h = cost_function(prediction, family_size, family_cost_matrix, accounting_cost_matrix)
    return (fc + ac) + (l + h) * 1000000

get_cost_consolidated(prediction)

77246.7745397196

In [10]:
prediction = pd.Series(prediction, name="assigned_day")
prediction.index.name = "family_id"
prediction.to_csv("submission.csv", index=True, header=True)